In [ ]:
!pip install datasets
from datasets import load_dataset
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the dataset
raw_datasets = load_dataset('iwslt2017', 'iwslt2017-zh-en')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/231266 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8549 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/879 [00:00<?, ? examples/s]

In [ ]:
# Sanity check
print(raw_datasets['train']['translation'][0])

{'en': "Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.", 'zh': '非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。'}


In [ ]:
subsets = ['train', 'test', 'validation']

for subset in subsets:
    english_sentences = []
    chinese_sentences = []

    # Assuming each translation is a dictionary with 'en' and 'zh' keys
    for translation in raw_datasets[subset]['translation']:
        english_sentences.append(translation['en'])
        chinese_sentences.append(translation['zh'])

    # Create a DataFrame
    df = pd.DataFrame({
        'en': english_sentences,
        'zh': chinese_sentences
    })

    # Save the DataFrame to a CSV file
    df.to_csv(f'{subset}.csv', index=False)

In [ ]:
# Sanity check
train_df = pd.read_csv('train.csv')
print(train_df['en'])  # All English sentences from the training set
print(train_df['zh'])  # All Chinese sentences from the training set

0         Thank you so much, Chris. And it's truly a gre...
1         I have been blown away by this conference, and...
2         And I say that sincerely, partly because  I ne...
3                  I flew on Air Force Two for eight years.
4         Now I have to take off my shoes or boots to ge...
                                ...                        
231261                            It's a tiny jungle party.
231262                   This forest grows as a collective.
231263    If the same trees -- same species -- would hav...
231264    And this is how we create a 100-year-old fores...
231265                                 Thank you very much.
Name: en, Length: 231266, dtype: object
0         非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。
1          这个会议真是让我感到惊叹不已，我还要谢谢你们留下的 关于我上次演讲的精彩评论
2          我是非常真诚的，部分原因是因为----我的确非常需要！ 你设身处地为我想想！
3                                     我坐了8年的空军二号。
4                               不过现在上飞机前我则要脱掉我的鞋子
                           ...                   


In [ ]:
val_df = pd.read_csv('validation.csv')
val_df

en  \
0    Last year I showed these two slides so that  d...   
1    But this understates the seriousness of this p...   
2    The arctic ice cap is, in a sense,  the beatin...   
3        It expands in winter and contracts in summer.   
4    The next slide I show you will be  a rapid fas...   
..                                                 ...   
874  You increase paralysis, and you decrease satis...   
875                        Everybody needs a fishbowl.   
876  This one is almost certainly too limited --  p...   
877  But the absence of some metaphorical fishbowl ...   
878                               Thank you very much.   

                                                    zh  
0    去年我给各位展示了两个 关于北极冰帽的演示 在过去三百万年中 其面积由相当于美国南方48州面...  
1                      但这些没能完全说明这个问题的严重性 因为这没有表示出冰帽的厚度  
2                             感觉上，北极冰帽 就好象全球气候系统中跳动的心脏  
3                                        冬天心脏舒张，夏天心脏收缩  
4                                下面我要展示的是 在过去25年里的极剧变化  
..                                                 ...  
874                                  你增加了的是瘫痪，减少了的是满足。  
875                                     每个人都需要这么个“鱼缸”。  
876                至于这个（鱼缸）嘛，对这条鱼 可能是小了点，对于我们几乎肯定是太小了。  
877                  但是，没有这么个象征性的鱼缸那就 意味着苦难将至， 也许是，灾难。  
878                                              非常感谢。  

[879 rows x 2 columns]

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

/Users/lauralee/anaconda3/envs/cs4248project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name = 'utrobinmv/t5_translate_en_ru_zh_large_1024'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

prefix = 'translate to zh: '
src_text = prefix + "Съешь ещё этих мягких французских булок."

# translate Russian to Chinese
input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids)

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['再多吃这些柔和的法国杯子。']


In [ ]:
prefix = 'translate to zh: '
src_text = prefix + "i study computer science"

input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids)

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)

['我学习计算机科学']


In [ ]:
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
import time

In [ ]:
file_path_train = '/content/drive/MyDrive/data/train.csv'
file_path_val = '/content/drive/MyDrive/data/validation.csv'

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
import pandas as pd

# Define your dataset class
class MyDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        input_text = row['en']
        target_text = row['zh']

        input_encoding = self.tokenizer(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        target_encoding = self.tokenizer(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'labels': target_encoding['input_ids'].flatten()
        }

# Load your fine-tuning data
train_df = pd.read_csv(file_path_train)
val_df = pd.read_csv(file_path_val)
# train_fraction =
# train_df, _ = train_test_split(train_df, train_size=train_fraction, random_state=42)

# Initialize model and tokenizer
model_name = 'utrobinmv/t5_translate_en_ru_zh_small_1024'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define your optimizer and other training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
batch_size = 32
epochs = 3
max_length = 32  # Define your desired max sequence length
consistency_weight = 0.1

# Create DataLoader objects
train_dataset = MyDataset(train_df, tokenizer, max_length)
val_dataset = MyDataset(val_df, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/809 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def translate(model, tokenizer, text, prefix):
    src_text = prefix + text
    input_ids = tokenizer(src_text, return_tensors="pt").to(device)
    generated_tokens = model.generate(**input_ids)
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]  # Access the first element of the list
    return result


In [ ]:
# Define self-consistency simply using cosine_similarity
# TODO: Test other metrics e.g. MSE, etc.
import torch
from torch.nn.functional import cosine_similarity

def self_consistency_loss(original_text, model, tokenizer):
    # Translate to Chinese
    translated_text = translate(model, tokenizer, original_text, 'translate to zh: ')

    # Translate back to English
    back_translated_text = translate(model, tokenizer, translated_text, 'translate to en: ')

    # Encoded texts
    encoded_input = tokenizer(original_text, return_tensors="pt").to(device)
    encoded_back_translated_input = tokenizer(back_translated_text, return_tensors="pt").to(device)

    # Calculate cosine similarity between original and back-translated text embeddings
    original_embedding = model.get_encoder()(**encoded_input).last_hidden_state.mean(dim=1)
    back_translated_embedding = model.get_encoder()(**encoded_back_translated_input).last_hidden_state.mean(dim=1)
    loss = 1 - cosine_similarity(original_embedding, back_translated_embedding)

    return loss

In [ ]:
# Fine-tuning loop
for epoch in range(epochs):
    model.train()
    epoch_start_time = time.time()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        translation_loss = outputs.loss

        # Self-consistency loss
        consistency_losses = []
        for input_id in input_ids:
            input_id = input_id.to(device)  # Move input_id to device
            text = tokenizer.decode(input_id, skip_special_tokens=True)
            loss = self_consistency_loss(text, model, tokenizer).to(device)
            consistency_losses.append(loss)
        consistency_loss = torch.mean(torch.stack(consistency_losses))

        # Total loss
        total_loss = translation_loss + consistency_weight * consistency_loss

        # Backward pass and optimization
        total_loss.backward()
        optimizer.step()


    # Validation
    model.eval()
    with torch.no_grad():
        total_loss = 0
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            total_loss += outputs.loss.item()

        avg_val_loss = total_loss / len(val_loader)
        remaining_epochs = epochs - (epoch + 1)

        print(f'Epoch {epoch + 1} - Validation Loss: {avg_val_loss:.4f}')

# Save the fine-tuned model
model.save_pretrained('fine_tuned_model')

In [ ]:
import sacrebleu
from jiwer import wer
# For TER, you might need to find an appropriate library or implement the calculation based on its formula.

In [ ]:
# Prepare for evaluation
prefix = 'translate to zh: '
predictions, references = [], []

for index, row in val_df.iterrows():
    src_text = prefix + row['en']
    input_ids = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids
    generated_tokens = model.generate(input_ids)
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    predictions.append(translation)
    references.append(row['zh'])

In [ ]:
references = [[reference] for reference in references]
print(references)


[['去年我给各位展示了两个 关于北极冰帽的演示 在过去三百万年中 其面积由相当于美国南方48州面积总和 缩减了40%'], ['但这些没能完全说明这个问题的严重性 因为这没有表示出冰帽的厚度'], ['感觉上，北极冰帽 就好象全球气候系统中跳动的心脏'], ['冬天心脏舒张，夏天心脏收缩'], ['下面我要展示的是 在过去25年里的极剧变化'], ['红色的是永冻冰'], ['你看，它正在变成深蓝色 这是每年冬天形成的年度冰 在夏天永冻冰收缩'], ['所谓的“永冻”，是指形成五年或更久的冰 你看，这也像血液一样 输送到身体各部位'], ['在25年的时间里，它从这里，到了这里'], ['值得注意的是 温室效应使得北冰洋周围的冻土层受热 而这里有大量被冻封的碳 解冻时，微生物降解碳形成甲烷'], ['如果突破顶点，温室气体排放量 将是现有大气层中的全球温室污染总量'], ['在阿拉斯加的一些浅湖里 已经可以看到水中探头的沼气泡'], ['去年冬天，University of Alaska的Katey Walter教授 结队去了一个浅湖'], ['哇（开心而又惊叹滴笑） 戈尔：她很好，我们怎么样呢？'], ['有一个原因，北方沉积的大量热能 加热了格陵兰岛'], ['这是一条每年融化的河'], ['但流量却比往年都要大'], ['这是格陵兰岛西南的Kangerlussuaq河'], ['如果你想了解陆地上的冰块融化 如何使得海平面上升 这里就是它的入海口'], ['这里的流量正在急速上升'], ['南极，地球的另一端 这个行星上最大的冰块'], ['上个月，科学家报告整个大洲 正处于冰量减少的阶段'], ['在南极洲的西部突然发现 几个低于海平面的岛屿正在加速融化'], ['这相当于海平面上20英尺，和格陵兰岛一样'], ['在Himalayas,第三大冰块 在顶部你可以看到新的湖泊，而几年前，这只是冰河'], ['全球40%从其融水中 获得一半的饮用水'], ['在安第斯山脉,这条冰河 是这座城市的饮用水源'], ['流量正在增加'], ['但当它们消失时，我们也将失去饮用水'], ['在California, Sierra的积雪 每年减少40%'], ['对于蓄水而言，这是一个打击'], ['而且如你看到的，预计是非常严重的'], ['全球的干燥化 正在导致火灾

In [ ]:
# Undo the transformation
references = [reference[0] for reference in references]
print(references)

['去年我给各位展示了两个 关于北极冰帽的演示 在过去三百万年中 其面积由相当于美国南方48州面积总和 缩减了40%', '但这些没能完全说明这个问题的严重性 因为这没有表示出冰帽的厚度', '感觉上，北极冰帽 就好象全球气候系统中跳动的心脏', '冬天心脏舒张，夏天心脏收缩', '下面我要展示的是 在过去25年里的极剧变化', '红色的是永冻冰', '你看，它正在变成深蓝色 这是每年冬天形成的年度冰 在夏天永冻冰收缩', '所谓的“永冻”，是指形成五年或更久的冰 你看，这也像血液一样 输送到身体各部位', '在25年的时间里，它从这里，到了这里', '值得注意的是 温室效应使得北冰洋周围的冻土层受热 而这里有大量被冻封的碳 解冻时，微生物降解碳形成甲烷', '如果突破顶点，温室气体排放量 将是现有大气层中的全球温室污染总量', '在阿拉斯加的一些浅湖里 已经可以看到水中探头的沼气泡', '去年冬天，University of Alaska的Katey Walter教授 结队去了一个浅湖', '哇（开心而又惊叹滴笑） 戈尔：她很好，我们怎么样呢？', '有一个原因，北方沉积的大量热能 加热了格陵兰岛', '这是一条每年融化的河', '但流量却比往年都要大', '这是格陵兰岛西南的Kangerlussuaq河', '如果你想了解陆地上的冰块融化 如何使得海平面上升 这里就是它的入海口', '这里的流量正在急速上升', '南极，地球的另一端 这个行星上最大的冰块', '上个月，科学家报告整个大洲 正处于冰量减少的阶段', '在南极洲的西部突然发现 几个低于海平面的岛屿正在加速融化', '这相当于海平面上20英尺，和格陵兰岛一样', '在Himalayas,第三大冰块 在顶部你可以看到新的湖泊，而几年前，这只是冰河', '全球40%从其融水中 获得一半的饮用水', '在安第斯山脉,这条冰河 是这座城市的饮用水源', '流量正在增加', '但当它们消失时，我们也将失去饮用水', '在California, Sierra的积雪 每年减少40%', '对于蓄水而言，这是一个打击', '而且如你看到的，预计是非常严重的', '全球的干燥化 正在导致火灾数量急剧增加', '而全世界的灾害数量 也正以绝对显著的 空前的速度增加', '在过去三十年内，灾害总数达到了 更早七十五年总数的

In [ ]:
# BLEU
bleu_score = sacrebleu.corpus_bleu(predictions, [references]).score
# WER
wer_score = sum([wer(ref, hyp) for ref, hyp in zip(references, predictions)]) / len(references)

In [ ]:
predictions_processed = []
for prediction in predictions:
    prediction = prediction.replace(" ", "")
    # prediction = prediction.replace(',', '，')
    predictions_processed.append(prediction)

In [ ]:
bleu_metric = sacrebleu.BLEU()
bleu_score = bleu_metric.corpus_score(predictions_processed, references)
print(f"BLEU Score: {bleu_score.score}")

BLEU Score: 6.742555929751843


In [ ]:
#k-smoothing
bleu_metric = sacrebleu.BLEU(smooth_method='exp', smooth_value=1)  # Adjust the smoothing value as needed
bleu_score = bleu_metric.corpus_score(predictions, references)
print(f"BLEU Score: {bleu_score.score}")

BLEU Score: 6.742555929751843


In [ ]:
bleu_score = sacrebleu.corpus_bleu(predictions, references).score
print(f"BLEU Score: {bleu_score}")


BLEU Score: 6.742555929751843


In [ ]:
bleu_score = sacrebleu.corpus_bleu(predictions, references).score
print(f"BLEU Score: {bleu_score}")


BLEU Score: 6.742555929751843


In [ ]:
bleu_metric = sacrebleu.BLEU()
bleu_score = bleu_metric.corpus_score(predictions, references)
print(f"BLEU Score: {bleu_score.score}")

BLEU Score: 6.742555929751843


In [ ]:
corpus_bleu = nltk.translate.bleu_score.corpus_bleu(references, predictions)
print(f"BLEU Score: {corpus_bleu}")

In [ ]:
import editdistance

In [ ]:
ter_score = sum([editdistance.eval(ref.split(), hyp.split()) / max(len(ref.split()), len(hyp.split())) for ref, hyp in zip(references, predictions)]) / len(references)


TER Score: 0.9980250683591801


In [ ]:
print(f"BLEU Score: {bleu_score}")
print(f"WER Score: {wer_score}")
print(f"TER Score: {ter_score}")

BLEU Score: 0.23399699282404335
WER Score: 1.047602510257441
TER Score: 0.9980250683591801


In [ ]:
import jieba

In [ ]:
# Prepare for evaluation
prefix = 'translate to zh: '
predictions_a, references_a = [], []

for index, row in val_df.iterrows():
    src_text_a = prefix + row['en']

    # Use Jieba tokenizer instead of the previous tokenizer
    tokens_a = list(jieba.cut(src_text_a))

    # Convert tokens back to a sentence
    input_text_a = ' '.join(tokens_a)

    # Now, you can continue with the rest of your code

    # Correct the order of these lines
    input_ids_a = tokenizer(input_text_a, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids
    generated_tokens_a = model.generate(input_ids_a)
    translation_a = tokenizer.batch_decode(generated_tokens_a, skip_special_tokens=True)[0]
    predictions_a.append(translation_a)
    references_a.append(row['zh'])

In [ ]:
bleu_metric = sacrebleu.BLEU()
bleu_score = bleu_metric.corpus_score(predictions_a, references_a)
print(f"BLEU Score: {bleu_score.score}")

BLEU Score: 0.0


In [ ]:
import editdistance

# Assuming predictions_a and references_a are lists of translated sentences
ter_score = sum([editdistance.eval(ref.split(), hyp.split()) / max(len(ref.split()), len(hyp.split())) for ref, hyp in zip(references_a, predictions_a)]) / len(references_a)

print(f"TER Score: {ter_score}")

TER Score: 0.9992415623814942


In [ ]:
# wno jie ba

# Assuming predictions_a and references_a are lists of translated sentences
ter_score = sacrebleu.corpus_ter(predictions,references)

print(f"TER Score: {ter_score}")

TER Score: TER = 36.28


In [ ]:
import editdistance

def ter(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Calculate Levenshtein distance
    distance = editdistance.eval(reference_words, hypothesis_words)

    # Normalize by the length of the reference sentence
    ter_score = distance / len(reference_words)

    return ter_score

# Assuming references_a and predictions_a are lists of translated sentences
ter_scores = [ter(ref, hyp) for ref, hyp in zip(references_a, predictions_a)]

# Calculate average TER score
average_ter_score = sum(ter_scores) / len(ter_scores)

print(f"Average TER Score: {average_ter_score}")


Average TER Score: 1.316639579608863


In [ ]:
import editdistance

def ter(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Calculate Levenshtein distance
    distance = editdistance.eval(reference_words, hypothesis_words)

    # Normalize by the length of the reference sentence
    ter_score = distance / len(reference_words)

    return ter_score

# Assuming references_a and predictions_a are lists of translated sentences
ter_scores = [ter(ref, hyp) for ref, hyp in zip(references, predictions)]

# Calculate average TER score
average_ter_score = sum(ter_scores) / len(ter_scores)

print(f"Average TER Score: {average_ter_score}")


Average TER Score: 1.047602510257441


In [ ]:
prefix = 'translate to zh: '
predictions_b, references_b = [], []

for index, row in val_df.iterrows():
    src_text = prefix + row['en']
    input_ids = tokenizer(src_text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids
    generated_tokens = model.generate(input_ids)
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    # Tokenize using Jieba
    predicted_tokens = list(jieba.cut(translation))
    reference_tokens = list(jieba.cut(row['zh']))

    # Join tokens back to form sentences
    predicted_sentence = ' '.join(predicted_tokens)
    reference_sentence = ' '.join(reference_tokens)

    predictions_b.append(predicted_sentence)
    references_b.append(reference_sentence)

KeyboardInterrupt: 

In [ ]:
print(references_b)
print(predictions_b)

In [ ]:
print(predictions)

NameError: name 'predictions' is not defined